In [1]:
import os 
import numpy as np
import torch
import torch.utils.data as Data
import torch.nn as nn
import torch.nn.functional as func
from torch.autograd import Function
from glob import glob
import pydicom
import cv2
import time
from skimage.transform import resize
import matplotlib.pyplot as plt


In [5]:
class hyperParam:
    def __init__(self):
        self.dataset_url = "/home/buchi/Desktop/buchi/data/dicom/dicom/"
        self.datasetMode = 1 #1 = InPhase, 2 = OutPhase, 3 = T2, 4 = In+OutPhase (Not Implimented yet)
        #self.interpFactor = None
        self.interpFactor = (0.5,0.5,0.5)
        self.runName = '3DWNetv2'
        self.combineLoss = False
        #network configure
        #self.ModelDownscale = True
        self.ModelDownscale = False
        self.InputCh=1
        self.ScaleRatio = 2
        self.ConvSize = 3
        self.pad = (self.ConvSize - 1) // 2 
        self.MaxLv = 5
        self.ChNum = [self.InputCh,64]
        for i in range(self.MaxLv-1):
            self.ChNum.append(self.ChNum[-1]*2)
        #data configure
        self.BatchSize = 1
        self.Shuffle = False
        self.LoadThread = 0
        self.inputsize = [224,224]
        #partition configure
        self.K = 10
        #training configure
        self.init_lr = 0.001
        self.lr_decay = 0.1
        self.lr_decay_iter = 5
        self.max_iter = 100
        self.checkpoint_frequency = 10
        self.cuda_dev = 0 
        self.cuda_dev_list = "4,5"
        self.check_iter = 1000
        self.useSSIMLoss = True
        #self.model_tested = "checkpoints/checkpoint_8_8_2_44_epoch_50"
        #self.model_tested = "/Users/dhanunjayamitta/Downloads/pretrained/checkpoint_3DWNetv2_SepLoss_ds2_ep100_epoch_100"
        self.model_tested = "checkpoints/checkpoint_9_16_17_8_epoch_990"
        #Ncuts Loss configure
        self.radius = 4
        self.sigmaI = 10
        self.sigmaX = 4

    def initiate(self):
        #pre-calculations
        if self.combineLoss:
            self.runName += '_CombLoss'
        else:
            self.runName += '_SepLoss'
        self.runName += '_ds'+str(self.datasetMode)+'_ep'+str(self.max_iter)
        

In [6]:
params = hyperParam()

In [9]:
"""
DATA STRUCTURE

dicom -> overall dataset folder
    scans -> training images 
        scans-001
        scans-002
        ...
        scans-050
        scans-051
    segmentations -> testing images
        segmentation-001
        segmentation-002
        ...
        segmentation-050
        segmentation-051

"""

In [ ]:
import os
def walk_through_dir(dir_path):
  """
  Walks through dir_path returning its contents.
  Args:
    dir_path (str or pathlib.Path): target directory
  
  Returns:
    A print out of:
      number of subdiretories in dir_path
      number of images (files) in each subdirectory
      name of each subdirectory
  """
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")